In [2]:
import torch
import json
import os
import numpy as np
from dsicl.utils import set_seed

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = '2'

加载一个模型

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
model_path = "../model/llama_1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
model = AutoModelForCausalLM.from_pretrained(model_path).half().to(torch.device('cuda'))


/home/gq/anaconda3/envs/DS-ICL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


读取数据集，用get_data_reader方法从一个json文件中读取，json文件格式要求如下：
{ \
   'data_info':{\
    'data_name': 'rte_train',\
    'label_space': ['entailment', 'not_entailment'],\
    'columns': ['premise', 'hypothesis', 'label']\
   },\
   'data':[\
    sample1,\
    sample2,\
    ...\
   ]\
}\
\
读取数据时传入label_map参数，将标签从默认标签转换为想要的标签

In [4]:
from src.data_reader import get_data_reader, DataReader, read_demo_benchmark
task = 'cr'
trainset, testset = read_demo_benchmark(task=task, seed=0)

In [5]:
testset[1]

{'sentence': "the extended zoom range and faster lense put it at the top of it 's class .",
 'label': 'positive'}

In [6]:
from src.prompter import Prompter
from src.template import DEMO_TEMPLATE, DEMO_HEAD
template = DEMO_TEMPLATE[task]
head = DEMO_HEAD[task]

prompter = Prompter(template=template, head=head, sep='\n\n')

In [7]:
print(prompter.generate_context(trainset[:2], testset[0]))

Classify the reviews based on whether their sentiment type is positive or negative.

Review:it 's not as stylized as a sony or samsung .
Sentiment:negative

Review:the 6600 will provide similar service in more developed areas of the states and not as well in more remote areas .
Sentiment:negative

Review:a ) feel cheap -- the plastic is feels like it would break very easily , and it definately wouldnt survive a drop
Sentiment:


In [8]:
from src.ranker import DEmORanker
set_seed(0)
original_demos = trainset.get_subset(8, balance=True)

In [9]:
original_demos

[{'sentence': 'unlike other reviewers , i have found the support to be responsive , almost no wait time on the phone , and relatively knowledgeable .',
  'label': 'positive'},
 {'sentence': "the best thing to do with this version , is to leave it on the store shelf . it doesn 't work , and will cost you an enormous amount of your time , not to mention your hard earned dollars .",
  'label': 'negative'},
 {'sentence': 'they have the worst service of any of the major companies .',
  'label': 'negative'},
 {'sentence': ') no locked-in proprietary business like apple ; . this baby plays wmas too .',
  'label': 'positive'},
 {'sentence': 'the manual is easy to understand , and it is mostly idiot proof .',
  'label': 'positive'},
 {'sentence': 'unfortunately , i sold it exactly a month later .',
  'label': 'negative'},
 {'sentence': "1 . the diaper champ doesn 't require special refills that are expensive and hard to find ! .",
  'label': 'positive'},
 {'sentence': 'the volume level of the p

In [10]:
ranker = DEmORanker(model, tokenizer, prompter, trainset.data_info['label_space'])

In [11]:
# demos_l = [ranker.rank(original_demos, d, seed=0) for d in testset]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


{'input_ids': tensor([[    1,  4134,  1598,   278, 21804,  2729,   373,  3692,  1009, 19688,
          1134,   338,  6374,   470,  8178, 29889,    13,    13,  1123,  1493,
         29901,   348,  4561,   916,  9076,   414,  1919,   474,   505,  1476,
           278,  2304,   304,   367,  5544,   573,  1919,  4359,   694,  4480,
           931,   373,   278,  9008,  1919,   322, 13774,  7134,   519,   869,
            13, 29903,   296,  2073, 29901,  1066,  3321,    13,    13,  1123,
          1493, 29901, 29896,   869,   278,  9766,   546, 18480,  1838,   525,
         29873,  1996,  4266,  2143,  6090,   393,   526, 19390,   322,  2898,
           304,  1284,  1738,   869,    13, 29903,   296,  2073, 29901,  1066,
          3321,    13,    13,  1123,  1493, 29901,  1552,  1900,  2655,   304,
           437,   411,   445,  1873,  1919,   338,   304,  5967,   372,   373,
           278,  3787,   528,   761,   869,   372,  1838,   525, 29873,   664,
          1919,   322,   674,  3438,  

AssertionError: 

初始化一个inferencer进行推理，目前支持direct inferencer（利用推测解码直接获取label上的概率）和generation inferencer（常规的自回归生成）

In [ ]:
from src.inferencer import DirectInferencer

labels = trainset.data_info['label_space']

direct_inferencer = DirectInferencer(model, tokenizer, prompter, labels)

对整个测试集推理

In [ ]:
# y_p = direct_inferencer.batch_infer(demos_l, testset)
# y_p[:5]

  0%|          | 0/256 [00:00<?, ?it/s]

100%|██████████| 256/256 [00:06<00:00, 42.64it/s]


['negative', 'positive', 'negative', 'negative', 'negative']

In [ ]:
direct_inferencer.infer(original_demos, testset[0])

'negative'

In [ ]:
baseline_yp = direct_inferencer.batch_infer([original_demos] * len(testset), testset)
baseline_yp[:5]

  0%|          | 0/256 [00:00<?, ?it/s]

100%|██████████| 256/256 [00:07<00:00, 36.47it/s]


['negative', 'negative', 'negative', 'negative', 'negative']

可以调用Evaluator类对结果进行评估（暂时只支持accuracy和f1-score，可以自己拓展）

In [ ]:
from src.evaluator import Evaluator
y_t = [testset[_]['label'] for _ in range(len(testset))]
evaluator = Evaluator()

In [ ]:
evaluator.acc_evaluate(baseline_yp, y_t)

0.5390625

In [ ]:
evaluator.acc_evaluate(y_p, y_t)

0.703125